In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline
import pandas as pd

/Users/andyho/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/andyho/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-10-29 13:23:38.131017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [35]:
from tqdm import tqdm
import numpy as np
import torch

In [4]:
filtered_philly_reviews = pd.read_csv('Data/filtered_philly_reviews.csv')
filtered_philly_reviews.head()

,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,...,categories,hours,review_id,user_id,stars_y,useful,funny,cool,text,date
0,0,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",n9pdoDkIip1pKzGzbvsegQ,8cu7AJE7gyq1-SLOcxUptA,5,0,0,0,A great neighborhood cafe! I especially love t...,2019-06-14
1,1,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",WcGtUuiXoeJDFxGxVEbPeA,P_9jCgiS7WTBjEU6SVG9gg,5,0,0,0,this place is so hipster! i ordered crepes whi...,2018-12-10
2,2,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",I9Dz9urftMCFFSKgAw5a-g,Te3BwftEFN8C9WxImhfM_g,5,2,0,2,What a gem of a place in the francisville neig...,2018-11-12
3,3,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",PJoin-6WYD7QzBZAhQS-Iw,zX6kC-irRdPn0Qs8pcYFKQ,5,0,0,0,This is the best pancake place in a city. Cozy...,2018-11-03
4,4,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,...,"Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",iZ_f9CY8WBDb0x95tSBSFg,a8qWaac2kpRAjl_6gHwJTw,5,0,0,0,This is my favorite café to go to whenever I'm...,2019-05-21


In [ ]:
#remove the first column from df
filtered_philly_reviews = filtered_philly_reviews.iloc[:,1:]

In [7]:
# Load Aspect-Based Sentiment Analysis model

large = "yangheng/deberta-v3-large-absa-v1.1"
base = "yangheng/deberta-v3-base-absa-v1.1"

absa_tokenizer = AutoTokenizer.from_pretrained(base)
absa_model = AutoModelForSequenceClassification \
  .from_pretrained(base)

# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

/Users/andyho/Library/Python/3.9/lib/python/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [32]:
coffee_keywords = ['coffee', 'espresso', 'latte', 'macchiato', 'flat white', 'pour over', 'cappuccino',
 'cold brew', 'cortado','mocha','americano']

# sentence = "The barista was rude and the overall vibes were off, and the pourover was astringent." \
#             "My partner got an oat flat white and she said was it was mediocre"

# sentence = "Always a good place to grab a hot latte. There might be pop up outside of the coffee shop. \
# The coffee here is really good! Got the brewed coffee and the vanilla latte. \
# The brewed coffee was very aromatic and came out hot. The vanilla latte was creamy and came out lukewarm. \
# It's pretty sweet which is what I prefer, but you can still really taste the coffee. Friendly staff, great service!"

# sentence = "Always a good place to grab a hot latte. I tried the lavender latte for the first time. \
# The flavor was good but it was too sweet for my liking."

# sentence = "The decor was really weird and not tasteful imo. The barista was really rude. Everything about the store \
#   was off. The oat latte was the best I've had though"

# sentence = "The almond croissant was dry and not flaky at all. The lavender latte was good though"

sentence = 'Such good iced coffee. Tastes like hot coffee tastes but actually is iced coffee. Some iced coffees are more bitter or not enough bitter but this one is the nice amount of bitter. And good ice to coffee ratio. Love the Halloween decorations too. Lovely.'


print(f"Sentence: {sentence}")
print()

# ABSA of coffee_keywords
aspect = coffee_keywords
inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
outputs = absa_model(**inputs)
probs = F.softmax(outputs.logits, dim=1)
probs = probs.detach().numpy()[0]
print(f"Sentiment of aspect '{aspect}' is:")
for prob, label in zip(probs, ["negative", "neutral", "positive"]):
  print(f"Label {label}: {prob}")
print()


sentiment = sentiment_model([sentence])[0]
print(f"Overall sentiment: {sentiment['label']} with score {sentiment['score']}")

Sentence: Such good iced coffee. Tastes like hot coffee tastes but actually is iced coffee. Some iced coffees are more bitter or not enough bitter but this one is the nice amount of bitter. And good ice to coffee ratio. Love the Halloween decorations too. Lovely.

Sentiment of aspect '['coffee', 'espresso', 'latte', 'macchiato', 'flat white', 'pour over', 'cappuccino', 'cold brew', 'cortado', 'mocha', 'americano']' is:
Label negative: 0.006518261507153511
Label neutral: 0.013929999433457851
Label positive: 0.9795517921447754

Overall sentiment: positive with score 0.7608826756477356


In [36]:
# Initialize a list to store the pss (positive sentiment score) for each review
pss_list = []
positive = []

# Iterate through each review in the DataFrame and calculate pss
for index, row in tqdm(filtered_philly_reviews.iterrows()):
    sentence = row['text']
    aspect = coffee_keywords
    inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
    outputs = absa_model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    pss = probs[0][2].item() # The third value in probs corresponds to the positive sentiment score
    pss_list.append(pss)
    probs_tensor = torch.tensor(probs, requires_grad=True)
    probs = probs.detach().numpy()
    if np.argmax(probs) == 2:
        positive.append(1)
    else:
            positive.append(0)


# Add the pss as a new column to the DataFrame
filtered_philly_reviews['pss'] = pss_list

# Add the positive as a new column to the DataFrame
filtered_philly_reviews['positive'] = positive

0it [00:00, ?it/s]/var/folders/2k/qbhytp6n0pnbr2rhg1w829yr0000gn/T/ipykernel_26186/642437726.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  probs_tensor = torch.tensor(probs, requires_grad=True)
2700it [55:15,  1.23s/it] 


In [44]:
# Group by business and calculate the average pss (Metric 1)
metric_1 = filtered_philly_reviews.groupby('business_id')['pss'].mean().reset_index()
metric_1.columns = ['business_id', 'metric_1']

In [45]:
metric_1

,business_id,metric_1
0,0Z8ps_OfcR9eJaV2JtxigA,0.671035
1,1w40T5J6br425rMvpS1zgQ,0.837065
2,2SevIwjgGdDdkIfs3Peu-Q,0.625708
3,41LEg4ZgFDVnZfToxGXiGA,0.718546
4,49Gyz-LD9MxTh1yzbYiOhw,0.729395
5,5Y7cqSgUSfLwn7ulBeO_sA,0.608716
6,5asvaAT-jDbOnCM3iPlScw,0.713236
7,6wqxuBt2InDipRuUj8BNuQ,0.749804
8,7cpj7hjDTX5fb-6pYcZiGQ,0.557076
9,99zoELPcbwRBUqiPoaPqiQ,0.897902


In [46]:
# Group by business and calculate the positive sentiment ratio (Metric 2)
metric_2 = filtered_philly_reviews.groupby('business_id')['positive'].mean().reset_index()
metric_2.columns = ['business_id', 'metric_2']

In [47]:
metric_2

,business_id,metric_2
0,0Z8ps_OfcR9eJaV2JtxigA,0.750000
1,1w40T5J6br425rMvpS1zgQ,0.920000
2,2SevIwjgGdDdkIfs3Peu-Q,0.711538
3,41LEg4ZgFDVnZfToxGXiGA,0.826087
4,49Gyz-LD9MxTh1yzbYiOhw,0.769231
5,5Y7cqSgUSfLwn7ulBeO_sA,0.705882
6,5asvaAT-jDbOnCM3iPlScw,0.821429
7,6wqxuBt2InDipRuUj8BNuQ,0.857143
8,7cpj7hjDTX5fb-6pYcZiGQ,0.606299
9,99zoELPcbwRBUqiPoaPqiQ,0.964286


In [48]:
# Merge metric_1 and metric_2 with the main DataFrame
filtered_philly_reviews = pd.merge(filtered_philly_reviews, metric_1, on='business_id')
filtered_philly_reviews = pd.merge(filtered_philly_reviews, metric_2, on='business_id')

In [51]:
# Compute the composite score
filtered_philly_reviews['composite_score'] = 0.6 * filtered_philly_reviews['metric_1'] + 0.4 * filtered_philly_reviews['metric_2']

In [72]:
# Group by business and calculate average composite score
business_scores = filtered_philly_reviews.groupby('business_id')['composite_score'].mean().reset_index()

# Sort by composite score descending
top_10_coffee_shops = business_scores.sort_values(by='composite_score', ascending=False).head(21)

# Merge with business information
top_10_coffee_shops = top_10_coffee_shops.merge(filtered_philly_reviews[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars_x', 'review_count', 'is_open', 'attributes', 'categories', 'hours','composite_score']].drop_duplicates(), on='business_id', how='left')


In [ ]:
#bold, thunder, pilgrim, grindcore, Konditori, Ray's Café & Teahouse, Knockbox Cafe, La Colombe Coffee, Vineyards Cafe, old city coffee

In [74]:
#drop rows that aren't coffee shops, i.e., restaurants that serve coffee or bakeries
top_10_coffee_shops = top_10_coffee_shops.drop([2,3,4,5,6,7,8,10,11,12,13])
top_10_coffee_shops

,business_id,composite_score_x,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,composite_score_y
0,99zoELPcbwRBUqiPoaPqiQ,0.924455,Bold Coffee Bar,1623 Ridge Ave,Philadelphia,PA,19130,39.969977,-75.163925,5.0,28,1,"{'RestaurantsReservations': 'False', 'OutdoorS...","Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",0.924455
1,dc98lqGzBSpZZExT_LSiCA,0.895554,Thunder Mug Cafe,4233 Ridge Ave,Philadelphia,PA,19129,40.009386,-75.194817,5.0,27,1,"{'BusinessParking': ""{'garage': False, 'street...","Coffee & Tea, Food","{'Monday': '8:0-15:0', 'Tuesday': '8:0-15:0', ...",0.895554
9,McQZweV9DLoNzbJWksPJ6w,0.835659,Pilgrim Roasters,4120 Main St,Philadelphia,PA,19127,40.022676,-75.219250,4.5,33,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForM...","Coffee & Tea, Coffee Roasteries, Food, Restaur...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ...",0.835659
14,9A5Gw0At6so0x-vWM0_JZw,0.802010,Grindcore House,1515 S 4th St,Philadelphia,PA,19147,39.929015,-75.151736,4.5,54,1,"{'BusinessParking': ""{'garage': False, 'street...","Coffee & Tea, Vegetarian, Food, Restaurants, V...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-16:0', '...",0.802010
15,T0qzsFxkHK9w1afnmns4fw,0.796863,Konditori,40 S 17th St,Philadelphia,PA,19103,39.951838,-75.168980,4.0,36,1,"{'OutdoorSeating': 'False', 'BusinessParking':...","Coffee & Tea, Juice Bars & Smoothies, Bakeries...","{'Monday': '6:30-20:0', 'Tuesday': '6:30-20:0'...",0.796863
16,Q9poenJ9SS3agpPo4Pgyyg,0.793714,Ray's Café & Teahouse,141 N 9th St,Philadelphia,PA,19107,39.954621,-75.154276,4.5,47,1,"{'WiFi': ""u'no'"", 'RestaurantsGoodForGroups': ...","Coffee & Tea, Chinese, Restaurants, Food","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",0.793714
17,QTjtz-bfm8Yrrv6t0PP0FQ,0.793423,Knockbox Cafe,405 S 45th St,Philadelphia,PA,19104,39.951443,-75.212341,4.0,25,1,"{'OutdoorSeating': 'True', 'WiFi': ""u'free'"", ...","Coffee & Tea, Bagels, Food, Restaurants, Sandw...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...",0.793423
18,6wqxuBt2InDipRuUj8BNuQ,0.792740,La Colombe Coffee,1414 South Penn Square,Philadelphia,PA,19102,39.951543,-75.165072,4.0,42,1,"{'BusinessParking': ""{'valet': False, 'garage'...","Coffee & Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '7:0-18:0', '...",0.792740
19,GOSVLhjUT6TnrPhkkmDOpw,0.769070,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,1,"{'RestaurantsReservations': 'False', 'Restaura...","Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '...",0.769070
20,h_rcMQtglIiAs-Oc5d8Ozw,0.768830,Old City Coffee,1136 Arch St,Philadelphia,PA,19107,39.953137,-75.159282,3.5,46,1,"{'OutdoorSeating': 'True', 'Alcohol': ""u'none'...","Coffee & Tea, Food, Coffee Roasteries","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:0', '...",0.768830


In [75]:
#rename composite score column and drop last column
top_10_coffee_shops = top_10_coffee_shops.rename(columns={"composite_score_x":"composite_score"})
top_10_coffee_shops = top_10_coffee_shops.iloc[:, :-1]
top_10_coffee_shops

,business_id,composite_score,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours
0,99zoELPcbwRBUqiPoaPqiQ,0.924455,Bold Coffee Bar,1623 Ridge Ave,Philadelphia,PA,19130,39.969977,-75.163925,5.0,28,1,"{'RestaurantsReservations': 'False', 'OutdoorS...","Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."
1,dc98lqGzBSpZZExT_LSiCA,0.895554,Thunder Mug Cafe,4233 Ridge Ave,Philadelphia,PA,19129,40.009386,-75.194817,5.0,27,1,"{'BusinessParking': ""{'garage': False, 'street...","Coffee & Tea, Food","{'Monday': '8:0-15:0', 'Tuesday': '8:0-15:0', ..."
9,McQZweV9DLoNzbJWksPJ6w,0.835659,Pilgrim Roasters,4120 Main St,Philadelphia,PA,19127,40.022676,-75.219250,4.5,33,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForM...","Coffee & Tea, Coffee Roasteries, Food, Restaur...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
14,9A5Gw0At6so0x-vWM0_JZw,0.802010,Grindcore House,1515 S 4th St,Philadelphia,PA,19147,39.929015,-75.151736,4.5,54,1,"{'BusinessParking': ""{'garage': False, 'street...","Coffee & Tea, Vegetarian, Food, Restaurants, V...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-16:0', '..."
15,T0qzsFxkHK9w1afnmns4fw,0.796863,Konditori,40 S 17th St,Philadelphia,PA,19103,39.951838,-75.168980,4.0,36,1,"{'OutdoorSeating': 'False', 'BusinessParking':...","Coffee & Tea, Juice Bars & Smoothies, Bakeries...","{'Monday': '6:30-20:0', 'Tuesday': '6:30-20:0'..."
16,Q9poenJ9SS3agpPo4Pgyyg,0.793714,Ray's Café & Teahouse,141 N 9th St,Philadelphia,PA,19107,39.954621,-75.154276,4.5,47,1,"{'WiFi': ""u'no'"", 'RestaurantsGoodForGroups': ...","Coffee & Tea, Chinese, Restaurants, Food","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
17,QTjtz-bfm8Yrrv6t0PP0FQ,0.793423,Knockbox Cafe,405 S 45th St,Philadelphia,PA,19104,39.951443,-75.212341,4.0,25,1,"{'OutdoorSeating': 'True', 'WiFi': ""u'free'"", ...","Coffee & Tea, Bagels, Food, Restaurants, Sandw...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
18,6wqxuBt2InDipRuUj8BNuQ,0.792740,La Colombe Coffee,1414 South Penn Square,Philadelphia,PA,19102,39.951543,-75.165072,4.0,42,1,"{'BusinessParking': ""{'valet': False, 'garage'...","Coffee & Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '7:0-18:0', '..."
19,GOSVLhjUT6TnrPhkkmDOpw,0.769070,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,1,"{'RestaurantsReservations': 'False', 'Restaura...","Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."
20,h_rcMQtglIiAs-Oc5d8Ozw,0.768830,Old City Coffee,1136 Arch St,Philadelphia,PA,19107,39.953137,-75.159282,3.5,46,1,"{'OutdoorSeating': 'True', 'Alcohol': ""u'none'...","Coffee & Tea, Food, Coffee Roasteries","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:0', '..."


In [76]:
#reset index
top_10_coffee_shops.reset_index(drop=True)

,business_id,composite_score,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours
0,99zoELPcbwRBUqiPoaPqiQ,0.924455,Bold Coffee Bar,1623 Ridge Ave,Philadelphia,PA,19130,39.969977,-75.163925,5.0,28,1,"{'RestaurantsReservations': 'False', 'OutdoorS...","Coffee & Tea, Food, Cafes, Restaurants","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."
1,dc98lqGzBSpZZExT_LSiCA,0.895554,Thunder Mug Cafe,4233 Ridge Ave,Philadelphia,PA,19129,40.009386,-75.194817,5.0,27,1,"{'BusinessParking': ""{'garage': False, 'street...","Coffee & Tea, Food","{'Monday': '8:0-15:0', 'Tuesday': '8:0-15:0', ..."
2,McQZweV9DLoNzbJWksPJ6w,0.835659,Pilgrim Roasters,4120 Main St,Philadelphia,PA,19127,40.022676,-75.219250,4.5,33,1,"{'RestaurantsGoodForGroups': 'True', 'GoodForM...","Coffee & Tea, Coffee Roasteries, Food, Restaur...","{'Monday': '7:0-15:0', 'Tuesday': '7:0-15:0', ..."
3,9A5Gw0At6so0x-vWM0_JZw,0.802010,Grindcore House,1515 S 4th St,Philadelphia,PA,19147,39.929015,-75.151736,4.5,54,1,"{'BusinessParking': ""{'garage': False, 'street...","Coffee & Tea, Vegetarian, Food, Restaurants, V...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-16:0', '..."
4,T0qzsFxkHK9w1afnmns4fw,0.796863,Konditori,40 S 17th St,Philadelphia,PA,19103,39.951838,-75.168980,4.0,36,1,"{'OutdoorSeating': 'False', 'BusinessParking':...","Coffee & Tea, Juice Bars & Smoothies, Bakeries...","{'Monday': '6:30-20:0', 'Tuesday': '6:30-20:0'..."
5,Q9poenJ9SS3agpPo4Pgyyg,0.793714,Ray's Café & Teahouse,141 N 9th St,Philadelphia,PA,19107,39.954621,-75.154276,4.5,47,1,"{'WiFi': ""u'no'"", 'RestaurantsGoodForGroups': ...","Coffee & Tea, Chinese, Restaurants, Food","{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ..."
6,QTjtz-bfm8Yrrv6t0PP0FQ,0.793423,Knockbox Cafe,405 S 45th St,Philadelphia,PA,19104,39.951443,-75.212341,4.0,25,1,"{'OutdoorSeating': 'True', 'WiFi': ""u'free'"", ...","Coffee & Tea, Bagels, Food, Restaurants, Sandw...","{'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ..."
7,6wqxuBt2InDipRuUj8BNuQ,0.792740,La Colombe Coffee,1414 South Penn Square,Philadelphia,PA,19102,39.951543,-75.165072,4.0,42,1,"{'BusinessParking': ""{'valet': False, 'garage'...","Coffee & Tea, Food","{'Monday': '0:0-0:0', 'Tuesday': '7:0-18:0', '..."
8,GOSVLhjUT6TnrPhkkmDOpw,0.769070,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,1,"{'RestaurantsReservations': 'False', 'Restaura...","Coffee & Tea, Cafes, Food, Creperies, Internet...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-15:0', '..."
9,h_rcMQtglIiAs-Oc5d8Ozw,0.768830,Old City Coffee,1136 Arch St,Philadelphia,PA,19107,39.953137,-75.159282,3.5,46,1,"{'OutdoorSeating': 'True', 'Alcohol': ""u'none'...","Coffee & Tea, Food, Coffee Roasteries","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:0', '..."


In [80]:
philly_reviews_asba.to_csv('philly_reviews_asba.csv')
top_10_coffee_shops.to_csv('top_10_coffee_shops.csv')

In [78]:
philly_reviews_asba = filtered_philly_reviews
philly_reviews_asba

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,useful,funny,cool,text,date,pss,positive,metric_1,metric_2,composite_score
0,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,A great neighborhood cafe! I especially love t...,2019-06-14,0.542547,1,0.735117,0.82,0.769070
1,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,this place is so hipster! i ordered crepes whi...,2018-12-10,0.959108,1,0.735117,0.82,0.769070
2,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,2,0,2,What a gem of a place in the francisville neig...,2018-11-12,0.839314,1,0.735117,0.82,0.769070
3,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,This is the best pancake place in a city. Cozy...,2018-11-03,0.362641,0,0.735117,0.82,0.769070
4,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,This is my favorite café to go to whenever I'm...,2019-05-21,0.720251,1,0.735117,0.82,0.769070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,2,0,0,This is a great cafe but I wish they served co...,2020-01-06,0.498637,1,0.671035,0.75,0.702621
2696,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,0,0,0,This is probably one of my favorite places in ...,2020-02-18,0.919786,1,0.671035,0.75,0.702621
2697,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,3,1,3,"Love the atmosphere, feels like a place you ca...",2019-12-03,0.676752,1,0.671035,0.75,0.702621
2698,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,2,1,1,One of my go-to's when I need somewhere in Cen...,2019-06-13,0.768508,1,0.671035,0.75,0.702621


In [77]:
filtered_philly_reviews

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,...,useful,funny,cool,text,date,pss,positive,metric_1,metric_2,composite_score
0,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,A great neighborhood cafe! I especially love t...,2019-06-14,0.542547,1,0.735117,0.82,0.769070
1,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,this place is so hipster! i ordered crepes whi...,2018-12-10,0.959108,1,0.735117,0.82,0.769070
2,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,2,0,2,What a gem of a place in the francisville neig...,2018-11-12,0.839314,1,0.735117,0.82,0.769070
3,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,This is the best pancake place in a city. Cozy...,2018-11-03,0.362641,0,0.735117,0.82,0.769070
4,GOSVLhjUT6TnrPhkkmDOpw,Vineyards Cafe,847 N 16th St,Philadelphia,PA,19130,39.969939,-75.163026,4.5,50,...,0,0,0,This is my favorite café to go to whenever I'm...,2019-05-21,0.720251,1,0.735117,0.82,0.769070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,2,0,0,This is a great cafe but I wish they served co...,2020-01-06,0.498637,1,0.671035,0.75,0.702621
2696,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,0,0,0,This is probably one of my favorite places in ...,2020-02-18,0.919786,1,0.671035,0.75,0.702621
2697,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,3,1,3,"Love the atmosphere, feels like a place you ca...",2019-12-03,0.676752,1,0.671035,0.75,0.702621
2698,0Z8ps_OfcR9eJaV2JtxigA,Square One Coffee,249 S 13th St,Philadelphia,PA,19107,39.947169,-75.162394,4.0,44,...,2,1,1,One of my go-to's when I need somewhere in Cen...,2019-06-13,0.768508,1,0.671035,0.75,0.702621
